In [4]:
# Variables and imports
import os
import requests
from itsdangerous import base64_encode
from dotenv import load_dotenv
from bs4 import BeautifulSoup

load_dotenv()

# From Spotify developer dashboard
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# Generated using local_server
refresh_token = os.getenv("SPOTIFY_REFRESH_TOKEN")

In [6]:
# Spotify authentication
def generate_access_token_from_refresh(ref_token):
    # Utility function to refresh the token
    headers = {
        "Authorization": "Basic " + base64_encode(f"{spotify_client_id}:{spotify_client_secret}").decode(),
        "Content-Type": "application/x-www-form-urlencoded"
    }
    r = requests.post(
        "https://accounts.spotify.com/api/token", data={
            "grant_type": "refresh_token",
            "refresh_token": ref_token,
        }, headers=headers)
    return r.json()["access_token"]

def get_spotify_data(url):
    return requests.get(url, headers={"Authorization": "Bearer " + access_token})


access_token = generate_access_token_from_refresh(refresh_token)

In [22]:
# Get Spotify playlists
r = get_spotify_data("https://api.spotify.com/v1/me/playlists?limit=10")
for playlist in r.json()["items"]:
    print(f"ID: {playlist['id']} \t {playlist['name']}")


ID: 2Y8UmzdFDZ0GcFDX6pCsQv 	 Rediscover - Mar 31st
ID: 2MknKWa2dMmVLz4fY1cymQ 	 Rediscover - Mar 25th
ID: 7nihSdgrqRK0cwwuidvlOf 	 Joseph Smith’s Secret Hymns
ID: 27B4P9xwJlmLtUSb6MgvsX 	 Rediscover - Mar 19th
ID: 1mlcQnCMGoB7YuC0N3yvI8 	 Rediscover - Mar 6th
ID: 15Rd1z8pbk2LF0VjFDTdsi 	 Rediscover - Feb 17th
ID: 3WSeteAA0wpziNg4tyWcYy 	 Rediscover - Feb 11th
ID: 6TaUOfDa9cdeY5DlmVfqUz 	 Rediscover - Feb 5th
ID: 7INyDT9gVnpVibIJiwXou4 	 Rediscover - Jan 6th
ID: 0PbHxgb8ciEcY8wLDunuqV 	 Rediscover - Jan 21st


In [7]:
# Choose Spotify playlist
spotify_playlist_id = "7nihSdgrqRK0cwwuidvlOf"

In [8]:
# Grab song data, save it
raw_song_data = get_spotify_data("https://api.spotify.com/v1/playlists/" + spotify_playlist_id)
songs = raw_song_data.json()["tracks"]["items"]


In [10]:
# Scrape lyric data from genius
# Source: https://medium.com/swlh/how-to-leverage-spotify-api-genius-lyrics-for-data-science-tasks-in-python-c36cdfb55cf3
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-'))
    songname2 = str(songname.replace(' ','-'))
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find("div", attrs={'data-lyrics-container' : True})
    if lyrics:
        return lyrics.get_text()
    return None

parsed_songs = []
for song in songs:
    # Greedy, but seems to work
    artist = song["track"]["artists"][0]["name"]
    song_name = song["track"]["name"]
    lyrics = scrape_lyrics(artist, song_name)
    parsed_songs.append({
        "artist": artist,
        "song_name": song_name,
        "lyrics": lyrics
    })


In [ ]:
# Convert songs to processable entity by openai

In [ ]:
# Save as JSONL document

In [ ]:
# Commands to run in command line to train the model

In [ ]:
# Run query against OpenAI fine-tuned